# Sisteme Multi-Agent: Dilema prizonierului
 - Andrei Olaru
 - Tudor Berariu
 


#### Scopul laboratorului

Scopul acestui laborator este de a vă familiariza cu noțiunea de [agent software](https://en.wikipedia.org/wiki/Software_agent). O parte din preocupările din domeniul sistemelor multi-agent se axează pe probleme din teoria jocurilor și modul cum agenții pot juca jocuri teoretice cât mai bine.

#### Dilema prizonierului

Dilema prizonierului (vezi și [aici](https://en.wikipedia.org/wiki/Prisoner's_dilemma)) este un joc de doi jucători în care fiecare jucător are la dispoziție două acțiuni: cooperarea cu celălalt jucător sau trădarea acestuia. În funcție de acțiunile alese, fiecare dintre cei doi jucători primește o recompensă. Recompensa agregată maximă este când ambii jucători cooperează, dar atracția este mai mare către trădare (vezi [matricea de joc](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma#Generalized_form)).

#### Dilema prizonierului, iterată

Dacă pentru un singur joc trădarea este opțiunea evidentă (cf. Echilibrului Nash), strategiile pot deveni mai complexe atunci când se joacă mai multe jocuri între aceiași doi jucători (poate exista ideea unei cooperări, pentru a crește scorul ambilor agenți).

#### Cerință

Se cere implementarea a 4 strategii 'standard' (vezi document pdf) pentru varianta iterată a jocului, împreună cu o **strategie proprie** care să se comporte acceptabil împotriva celorlalte, într-un turneu. Ceea ce ne interesează este **scorul** total obținut, mai ales raportat la numărul de jocuri (score/games).

In [85]:
from random import choice, randint

D = 'Defect'
C = 'Cooperate'

rewards = {(C, C): (4, 4), (C, D): (0, 5), (D, C): (5, 0), (D, D): (1, 1)}

In [86]:
# O strategie de joc este caracterizată de o funcție care întoarce, la fiecare apel, acțiunea aleasă de strategie.
# Parametrul primit de funcție este o listă de tupluri care conține acțiunile jucate anterior în cursul aceluiași 
# joc iterat.
# Fiecare tuplu din listă corespunde unui joc individual de dilema prizonierului și conține pe prima poziție 
# acțiunea aleasă de acest agent și pe a doua acțiunea jucată de oponent.

def AllD(_):
    
    return D

def Random(_):
    if randint(0, 100000) % 2 == 0:
        return D
    else:
        return C

ME = 0
ENEMY = 1

def TFT(information):
    # TODO
    if information == []:
        return C
    return information[-1][ENEMY]

no_cooperate = 0
def Joss(information):
    # TODO
    global no_cooperate
    
    if information == []:
        no_cooperate == 0
        return C
    if information[-1][ENEMY] == C:
       no_cooperate += 1
       if no_cooperate == 9:
            no_cooperate = 0
            return D
       else:
           return C
           
    else:    
        return D
    
test_tester = 0
def Tester(information):
    global test_tester
    
    if information == []:
        test_tester = 0
        return D
    
    for action in information:
        if action[ENEMY] == D:
            return TFT(information)
    
    if test_tester < 2:
        test_tester += 1
        return C
    else:
        test_tester = 0
        return D

# TODO: o nouă strategie
is_trader = False
def grudger(information):
    global is_trader
    if information == []:
        is_trader = False
        return C
    
    if is_trader:
        return D
    
    if information[-1][ENEMY] == D:
        is_trader = True
    
    return C

def all_c(_):
    return C

tftt = 0
def TFTT(infomation):
    global tftt
    if infomation == []:
        tftt = 0
        return C
    
    if infomation[-1][ENEMY] == D:
        tftt += 1
    
    if tftt == 2:
        # tftt = 0
        return D
    
    return C

def invers(infomation):
    
    if infomation == []:
        return D
    
    if infomation[-1][ENEMY] == D:
        return C
    else:
        return D

In [87]:
# TODO de activat strategiile aici
availableStrategies = [
    ('All-D', AllD),
    ('Random', Random),
    ('Tit-For-Tat', TFT), 
    ('Joss', Joss),
    ('Tester', Tester),
    ("Grudger", grudger),
    ("All-C", all_c),
    ("TITFORTWOTATS",TFTT ),
    ("Invers", invers)
]

In [88]:
strategies = []
for (name, proc) in availableStrategies:
    strategies.append({'name': name, 'procedure': proc, 'wins': 0, 'score': 0, 'games': 0, 'plays': {}})

# joacă un joc între A și B, întoarce recompensele asociate
def play_game(players, verbose = False):
    choices = [p['strategy']['procedure'](p['information']) for p in players]
    for i in range(2):
        players[i]['information'].append((choices[i], choices[1 - i]))
    if verbose: print(players[0]['strategy']['name']+" vs "+players[1]['strategy']['name']+" choices: "+str(choices)+" rewards: "+str(rewards[tuple(choices)]))
    return rewards[tuple(choices)]
    
# joacă `iterations` jocuri între A și B, întorcând scorul asociat întregului joc iterat
def play_iterated_pd(players, n_iterations, verbose = False):
    score = (0, 0)
    for i in range(n_iterations):
        rewardsAB = play_game(players, verbose)
        score = tuple([score[pi] + rewardsAB[pi] for pi in range(2)])
    if verbose: print("== result: "+str(score))
    return score

# joacă un turneu de n jocuri de câte n iterații, alegând aleator între strategiile date în `strategies`
def tournament(n_games, n_iterations, strategies, verbose = False):
    for game in range(n_games):
        agents = []
        strat = []
        for i in range(2):
            agents.append({'strategy': choice(strategies), 'information': []})
            strat.append(agents[i]['strategy'])
        for i in range(2):
            for j in range(2):
                if i != j:
                    if strat[j]['name'] not in strat[i]['plays']:
                        strat[i]['plays'][strat[j]['name']] = 1
                    else:
                        strat[i]['plays'][strat[j]['name']] += 1
        scores = play_iterated_pd(agents, n_iterations, verbose)
        result = (0, 0)
        if scores[0] > scores[1]:
            result = (1, 0)
        if scores[0] < scores[1]:
            result = (0, 1)
        for i in range(2):
            strat[i]['wins'] += result[i]
            strat[i]['score'] += scores[i]
            strat[i]['games'] += 1
    print('\n\n================ total games: ' + str(n_games))
    for s in strategies:
        print('\n strategy ' + s['name'])
        if s['games']:
            plays = ' played against '
            for s_op in strategies:
                if s_op['name'] in s['plays']:
                    plays += s_op['name'] + ' (' + str(s['plays'][s_op['name']]) + ') '
            print('\t' + plays)
            print( 
              '\t played '+str(s['games'])+' times and won '+str(s['wins'])+' times with a global score of '+str(s['score']) +
                  '\n\t score/games: '+str(round(float(s['score'])/s['games'], 2))+ 
                  '\t wins/games: '+str(round(float(s['wins'])/s['games'], 2))+
                  '\t score/wins: '+(str(round(float(s['score'])/s['wins'], 2)) if s['wins'] else "--"))
        else:
            print("\t played no games.")
        
        
# tournament(50, 10, strategies, True) # test, with Verbose
# tournament(10, 10, strategies) # short
tournament(500, 100, strategies) # short
tournament(5000, 200, strategies) # long



================ total games: 500

 strategy All-D
	 played against All-D (14) Random (8) Tit-For-Tat (12) Joss (17) Tester (7) Grudger (21) All-C (11) TITFORTWOTATS (5) Invers (15) 
	 played 110 times and won 89 times with a global score of 25228
	 score/games: 229.35	 wins/games: 0.81	 score/wins: 283.46

 strategy Random
	 played against All-D (8) Random (12) Tit-For-Tat (7) Joss (12) Tester (19) Grudger (13) All-C (15) TITFORTWOTATS (16) Invers (9) 
	 played 111 times and won 47 times with a global score of 29311
	 score/games: 264.06	 wins/games: 0.42	 score/wins: 623.64

 strategy Tit-For-Tat
	 played against All-D (12) Random (7) Tit-For-Tat (14) Joss (15) Tester (17) Grudger (9) All-C (11) TITFORTWOTATS (9) Invers (8) 
	 played 102 times and won 0 times with a global score of 28494
	 score/games: 279.35	 wins/games: 0.0	 score/wins: --

 strategy Joss
	 played against All-D (17) Random (12) Tit-For-Tat (15) Joss (12) Tester (13) Grudger (8) All-C (10) TITFORTWOTATS (10) Inver